<a href="https://colab.research.google.com/github/AndrewAOh/Strike-Probability/blob/main/Updated_Strike_Probability_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Importing data

In [ ]:
import pandas as pd
import glob
import os
import numpy as np

df2 = pd.read_csv("/content/drive/Shareddrives/USC Baseball/Combined Data/23_NCAA_Dataset*.csv")

<ipython-input-2-5db35e9a50b0>:6: DtypeWarning: Columns (27,92,93,94,167) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv("/content/drive/Shareddrives/USC Baseball/Combined Data/23_NCAA_Dataset*.csv")


# Clean Up Dataset & Recode Variables

In [ ]:
cleanD1 = df2[df2['Level'] == "D1"]
df3 = cleanD1.dropna(subset=['PlateLocHeight', 'PlateLocSide'])

In [ ]:
nonSwings = df3[df3['PitchCall'].str.contains("InPlay|StrikeSwinging|FoulBall|BallIntentional|PitchCall|Undefined") == False]
nonSwings = nonSwings[nonSwings['TaggedPitchType'].str.contains("Undefined") == False]
df = nonSwings

In [ ]:
from sklearn.model_selection import train_test_split
import matplotlib as plt
from sklearn import datasets
from sklearn import metrics


# creates list of conditions that determine whether a pitch is a ball or strike
conditions = [
    (df['PitchCall'] == 'BallCalled'),
    (df['PitchCall'] == 'BallinDirt'),
    (df['PitchCall'] == 'HitByPitch'),
    (df['PitchCall'] == 'StrikeCalled')
]

# creates a list of values to assign for strike (1) or not (0)
values = [0, 0, 0, 1]
# creates new column determining strike or not and assign values to it based on the conditions
df['isStrike'] = np.select(conditions, values)

In [ ]:
# Recodes pitch types into 5 different buckets

# sinkers, 4 seam, 2 seam, 1 seam = fastball
# cutters = sliders
# splitter = changeup

# creates list of conditions that determine whether TaggedPitchType is fastball/slider/changeup/curveball
conditions = [
    (df['TaggedPitchType'].str.contains('Fastball|FourSeamFastBall|TwoSeamFastBall|OneSeamFastBall|Sinker')),  # fastballs
    (df['TaggedPitchType'].str.contains('Slider|Cutter')),  # sliders
    (df['TaggedPitchType'].str.contains('ChangeUp|Splitter')),  # changeups
    (df['TaggedPitchType'].str.contains('Curveball')) # curveballs
]

# creates a list of values to assign for pitch types
values = [0, 1, 2, 3]
# creates new column determining pitch type and assign values to it based on the conditions; assigns 4 if an other pitch
df['pitchType'] = np.select(conditions, values, default = 4)

In [ ]:
# Recodes Pitch Handedness to binary variable

conditions = [
    (df['PitcherThrows'] == 'Right'),  # RHP
    (df['PitcherThrows'] == 'Left')  # LHP
]

# creates a list of values to assign for pitch handedness where RHP = 1 & LHP = 0
values = [0, 1]
# creates new column determining pitch hand
df['pitcherHand'] = np.select(conditions, values)

In [ ]:
# Recodes Hitting Handedness to binary variable

conditions = [
    (df['BatterSide'] == 'Right'),  # RHB
    (df['BatterSide'] == 'Left')  # LHB
]

# creates a list of values to assign for hitter handedness where RHB = 1 & LHB = 0
values = [0, 1]
# creates new column determining hit hand
df['hitterHand'] = np.select(conditions, values)

# Following Code Identifies Important Variables

In [ ]:
df4 = df[["pitchType", "RelSpeed",	"VertRelAngle",	"HorzRelAngle",
          "SpinRate",	"SpinAxis",	"RelHeight", "RelSide", "Extension",
          "VertBreak", "InducedVertBreak", "HorzBreak",	"ZoneSpeed", "VertApprAngle",	"HorzApprAngle",
          "ZoneTime",	"pfxx",	"pfxz",	"x0",	"y0",	"z0",	"vx0", "vy0",	"vz0",
          "ax0", "ay0", "az0", "EffectiveVelo",	"SpeedDrop",
          "PitchTrajectoryXc0",	"PitchTrajectoryXc1",	"PitchTrajectoryXc2",
          "PitchTrajectoryYc0", "PitchTrajectoryYc1",	"PitchTrajectoryYc2",
          "PitchTrajectoryZc0",	"PitchTrajectoryZc1",
          "PitchTrajectoryZc2"]].dropna()
df4.corr()

,pitchType,RelSpeed,VertRelAngle,HorzRelAngle,SpinRate,SpinAxis,RelHeight,RelSide,Extension,VertBreak,...,SpeedDrop,PitchTrajectoryXc0,PitchTrajectoryXc1,PitchTrajectoryXc2,PitchTrajectoryYc0,PitchTrajectoryYc1,PitchTrajectoryYc2,PitchTrajectoryZc0,PitchTrajectoryZc1,PitchTrajectoryZc2
pitchType,1.000000,-0.728277,0.431059,0.069171,-0.075416,-0.244827,-0.007335,0.000847,-0.253244,-0.755445,...,-0.515186,0.254610,0.726251,-0.584192,-0.001238,0.426773,-0.704917,-0.001445,-0.082092,0.210374
RelSpeed,-0.728277,1.000000,-0.602283,-0.158926,0.058478,0.295966,0.165311,0.073732,0.315546,0.884628,...,0.636744,-0.316673,-0.999748,0.802549,0.158809,-0.603167,0.782446,-0.072911,0.179443,-0.333213
VertRelAngle,0.431059,-0.602283,1.000000,0.112735,0.061108,-0.195062,-0.402911,0.016032,-0.287908,-0.667490,...,-0.270016,0.288019,0.596615,-0.472437,-0.397682,0.996782,-0.689369,-0.016581,-0.124115,0.195564
HorzRelAngle,0.069171,-0.158926,0.112735,1.000000,-0.038156,-0.256894,0.058742,-0.843789,-0.087063,-0.093473,...,-0.082633,0.086980,0.155624,-0.127845,0.059462,0.117050,-0.063165,0.843020,-0.997979,0.703118
SpinRate,-0.075416,0.058478,0.061108,-0.038156,1.000000,-0.219592,0.034288,0.093732,-0.204996,-0.100394,...,-0.008972,0.205281,-0.061273,0.055642,0.035377,0.046299,-0.110005,-0.094233,0.044769,0.114332
SpinAxis,-0.244827,0.295966,-0.195062,-0.256894,-0.219592,1.000000,-0.061193,0.026084,0.167263,0.333337,...,0.227744,-0.167761,-0.293213,0.235876,-0.064019,-0.193262,0.320264,-0.023914,0.264665,-0.729756
RelHeight,-0.007335,0.165311,-0.402911,0.058742,0.034288,-0.061193,1.000000,-0.075435,0.015698,0.195100,...,0.034892,-0.015670,-0.169225,0.138661,0.999965,-0.406237,0.252778,0.075451,-0.050408,0.024911
RelSide,0.000847,0.073732,0.016032,-0.843789,0.093732,0.026084,-0.075435,1.000000,0.026128,-0.006336,...,0.028816,-0.025964,-0.073698,0.060618,-0.075290,0.011427,-0.042196,-0.999997,0.838289,-0.515964
Extension,-0.253244,0.315546,-0.287908,-0.087063,-0.204996,0.167263,0.015698,0.026128,1.000000,0.425245,...,0.192050,-0.999988,-0.312819,0.271865,0.012401,-0.282218,0.389549,-0.025710,0.090865,-0.161954
VertBreak,-0.755445,0.884628,-0.667490,-0.093473,-0.100394,0.333337,0.195100,-0.006336,0.425245,1.000000,...,0.616961,-0.426965,-0.882137,0.711430,0.187142,-0.656570,0.965763,0.007184,0.108969,-0.291521


In [ ]:
df6 = df4.corr()
csv = df6.to_csv("/content/drive/Shareddrives/USC Baseball/ML Stuff/PitcherCorr*.csv", index=False)

In [ ]:
import statistics

# 'target_column' is the name of the column with 0s and 1s, and 'predictor_columns' is a list of predictor column names.

def compute_statistics_by_target(df, target_column, predictor_columns):
    # Group the dataframe by the target variable and calculate statistics for each group
    grouped = df.groupby(target_column)

    # Initialize a dictionary to store the statistics for each group
    statistics_dict = {}

    for target_value, group in grouped:
        # Calculate the means for each predictor column
        means = group[predictor_columns].mean()

        # Calculate the standard deviations for each predictor column
        std_devs = group[predictor_columns].std()

        # Calculate the medians for each predictor column
        medians = group[predictor_columns].median()

        # Calculate the 25th percentile for each predictor column
        percentile_25 = group[predictor_columns].quantile(0.25)

        # Calculate the 75th percentile for each predictor column
        percentile_75 = group[predictor_columns].quantile(0.75)

        # Combine the means and standard deviations for each group into a single row
        row = means.tolist() + std_devs.tolist() + medians.tolist() + percentile_25.tolist() + percentile_75.tolist()

        # Add the row to the statistics dictionary
        statistics_dict[target_value] = row

    # Convert the dictionary to a dataframe and set the index
    statistics_df = pd.DataFrame.from_dict(statistics_dict, orient='index', columns=predictor_columns +
                                        [f'Std_Dev_{col}' for col in predictor_columns] +
                                        [f'Median_{col}' for col in predictor_columns] +
                                        [f'Percentile_25_{col}' for col in predictor_columns] +
                                        [f'Percentile_75_{col}' for col in predictor_columns])


    return statistics_df

# Example usage:
target_column_name = 'isStrike'
predictor_columns_list = ["RelSpeed",	"VertRelAngle",	"HorzRelAngle",
          "SpinRate",	"SpinAxis",	"RelHeight", "RelSide", "Extension", "PlateLocHeight", "PlateLocSide",
          "VertBreak", "InducedVertBreak", "HorzBreak",	"ZoneSpeed", "VertApprAngle",	"HorzApprAngle",
          "ZoneTime",	"pfxx",	"pfxz",	"x0",	"y0",	"z0",	"vx0", "vy0",	"vz0",
          "ax0", "ay0", "az0", "EffectiveVelo",	"SpeedDrop",
          "PitchTrajectoryXc0",	"PitchTrajectoryXc1",	"PitchTrajectoryXc2",
          "PitchTrajectoryYc0", "PitchTrajectoryYc1",	"PitchTrajectoryYc2",
          "PitchTrajectoryZc0",	"PitchTrajectoryZc1",
          "PitchTrajectoryZc2"]
statistics_table = compute_statistics_by_target(df, target_column_name, predictor_columns_list)

print(statistics_table)


    RelSpeed  VertRelAngle  HorzRelAngle     SpinRate    SpinAxis  RelHeight  \
0  84.966826     -1.030562     -1.188618  2175.627872  177.145187   5.717201   
1  85.227330     -0.914174     -1.100188  2202.812413  175.815808   5.716591   

    RelSide  Extension  PlateLocHeight  PlateLocSide  ...  \
0  0.796327   5.747266        2.170383     -0.127437  ...   
1  0.798831   5.743921        2.331345     -0.050537  ...   

   Percentile_75_SpeedDrop  Percentile_75_PitchTrajectoryXc0  \
0                 8.110960                          55.14809   
1                 8.143475                          55.15385   

   Percentile_75_PitchTrajectoryXc1  Percentile_75_PitchTrajectoryXc2  \
0                       -117.395313                         14.183302   
1                       -117.646555                         14.243160   

   Percentile_75_PitchTrajectoryYc0  Percentile_75_PitchTrajectoryYc1  \
0                           6.06384                           0.21887   
1               

In [ ]:
csv = statistics_table.to_csv("/content/drive/Shareddrives/USC Baseball/ML Stuff/PitcherStrikeANOVA*.csv", index=False)

# Making the ML Model

In [ ]:
df['isStrike'].value_counts()

0    746592
1    333847
Name: isStrike, dtype: int64

In [ ]:
minority_class = df[df['isStrike'] == 1]
majority_class = df[df['isStrike'] == 0]

print("majority class is: " + str(len(majority_class)))
print("minority class is: " + str(len(minority_class)))

print("total is: " + str(len(majority_class)+len(minority_class)))

majority class is: 746592
minority class is: 333847
total is: 1080439


In [ ]:
# resamples the class to have a even distribution of strikes and balls
from sklearn.utils import resample
majority_class_downsampled = resample(majority_class,
                                      replace=False,  # sample without replacement
                                      n_samples=len(minority_class),  # match minority class size
                                      random_state=42)
# Combine minority class and downsampled majority class
balanced_df = pd.concat([minority_class, majority_class_downsampled])




In [ ]:
balanced_df = balanced_df.sample(frac=1, random_state=42)

In [ ]:
rel_data = balanced_df[['PlateLocHeight', 'PlateLocSide', 'pitchType', 'RelHeight', 'RelSide', 'hitterHand']].to_numpy()
classification = balanced_df['isStrike'].to_numpy()

In [ ]:
minority = balanced_df[balanced_df['isStrike'] == 1]
majority = balanced_df[balanced_df['isStrike'] == 0]

print("majority class is: " + str(len(majority)))
print("minority class is: " + str(len(minority)))

print("total is: " + str(len(majority)+len(minority)))

majority class is: 333847
minority class is: 333847
total is: 667694


In [ ]:
trainNum = int((len(majority) + len(minority))*0.7)

y = np.unique(classification, return_inverse=True)
y_train = y[1][:trainNum]
y_test = y[1][trainNum:]

x = np.unique(classification, return_inverse=True)
x_train = rel_data[:trainNum]
x_test = rel_data[trainNum:]

In [ ]:
from sklearn import preprocessing

scaler = preprocessing.StandardScaler().fit(x_train)
X_scaled = scaler.transform(x_train)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

names = [
    "Decision Tree",
    "Random Forest"
]

classifiers = [
    #DecisionTreeClassifier(max_depth=50),
    #DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=None, n_estimators=50)
]

In [ ]:
accuracies = []
for opt in classifiers:
  clf = make_pipeline(StandardScaler(), opt)
  clf.fit(x_train, y_train)

  y_pred = clf.predict((x_test))
  accuracies.append(metrics.accuracy_score(y_test, y_pred))

In [ ]:
from sklearn.metrics import classification_report
target_names = ['ball', 'strike']
print(classification_report(y_test, y_pred, target_names=target_names))

              precision    recall  f1-score   support

        ball       0.96      0.93      0.94    100057
      strike       0.93      0.96      0.95    100252

    accuracy                           0.94    200309
   macro avg       0.94      0.94      0.94    200309
weighted avg       0.94      0.94      0.94    200309



In [ ]:
print(accuracies)

[0.9443759391739762]


Visualizing Model

# Export Final Model

In [ ]:
from sklearn.tree import DecisionTreeClassifier
import joblib
from joblib import dump

import os
os.chdir('/content/drive/Shareddrives/USC Baseball/SabermetricModels')

# export the trained model to disk
joblib.dump(clf, 'StrikeProbModel.joblib')

['StrikeProbModel.joblib']